In [ ]:
import sys,os
sys.path.append("..")
import django
django.setup()
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import sqlite3
from crawlers.models import *

# 股價資料整批輸入

In [ ]:
# connect to finlab sqlite
conn = sqlite3.connect('/Users/benbilly3/Desktop/pythonTradeEye/Tool/data/data.db')

# connect to finlab_django mysql
connect_info = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format("root","benbilly3@","localhost",3306,"finlab_django_dev")
engine = create_engine(connect_info)
db = pymysql.connect("localhost", "root", "benbilly3@","finlab_django_dev", charset='utf8')


In [ ]:
# read data from sql

def read_sqlite_data(tableName):
    df = pd.read_sql('select * from'+' '+tableName,
                     conn,parse_dates=['date']).fillna(method='ffill')
    return df



# read_sqlite_data('price')

In [ ]:
#篩選掉股號奇怪的舊資料
def select_otc_id(code):
    if len(code) > 5:
        if code[-1] == "P":
            return False
        else:
            try:
                code = int(code)
                if code > 10000:
                    return False
                else:
                    return True
            except ValueError:
                return True
    else:
        return True
    
    
df=read_sqlite_data('price').loc[:,['stock_id','date','成交股數','成交金額','收盤價','最低價','最高價','開盤價','證券名稱']]
df2=df.rename(columns={'成交股數':'turnover_vol','成交金額':'turnover_price',
                      '收盤價':'close_price','最低價':'low_price',
                      '最高價':'high_price','開盤價':'open_price',
                      '證券名稱':'stock_name'})


df2

In [ ]:
df2["check"] = df2["stock_id"].apply(lambda s: select_otc_id(s))
df2=df2[df2["check"]]
df2=df2.iloc[:,:-1]

In [ ]:
df2=pd.read_csv('/Users/benbilly3/Downloads/brokerinfotw')
df2=df2.iloc[:,1:]
df2

# Add_to_sql 測試

In [ ]:
df2.to_sql(con=engine,name='crawlers_brokerinfotw',index_label='id', if_exists='replace')
                                                                
                                                                